1. ### Import packages

In [44]:
import pgeocode
import os
import pandas as pd

In [46]:
path = os.getcwd()

2. ### Import data - Pincode by locality in India 

In [51]:
path1 = path + '/Data/Locality_village_pincode_final_mar-2017.csv'
data = pd.read_csv(path1,encoding= 'unicode_escape')

In [53]:
data.head()

,Village/Locality name,Officename ( BO/SO/HO),Pincode,Sub-distname,Districtname,StateName
0,Aliganj,Lodi Road H.O,110003,Defence Colony,SOUTH EAST DELHI,DELHI
1,Kasturba Nagar,Lodi Road H.O,110003,Defence Colony,SOUTH EAST DELHI,DELHI
2,Jeewan Nagar,Jungpura S.O,110014,Defence Colony,SOUTH EAST DELHI,DELHI
3,Tehkhand,Okhla Industrial Estate S.O,110020,Defence Colony,SOUTH EAST DELHI,DELHI
4,Zakir Nagar SO,New Friends Colony S.O,110025,Defence Colony,SOUTH EAST DELHI,DELHI


3. ### Apply Filters

In [57]:
sorted(data['Districtname'][data['StateName']=='TAMIL NADU'].unique())

['ARIYALUR',
 'CHENNAI',
 'COIMBATORE',
 'CUDDALORE',
 'DHARMAPURI',
 'DINDIGUL',
 'ERODE',
 'KANCHIPURAM',
 'KANYAKUMARI',
 'KARUR',
 'KRISHNAGIRI',
 'MADURAI',
 'NAGAPATTINAM',
 'NAMAKKAL',
 'NILGIRIS',
 'PERAMBALUR',
 'PUDUKKOTTAI',
 'RAMANATHAPURAM',
 'SALEM',
 'SIVAGANGA',
 'THANJAVUR',
 'THENI',
 'TIRUCHIRAPPALLI',
 'TIRUNELVELI',
 'TIRUPPUR',
 'TIRUVALLUR',
 'TIRUVANNAMALAI',
 'TIRUVARUR',
 'TUTICORIN',
 'VELLORE',
 'VILLUPURAM',
 'VIRUDHUNAGAR']

In [141]:
district = ['CHENNAI','KANCHIPURAM']
data1 = data[data['Districtname'].isin(district)]

In [70]:
data1.head()

,Village/Locality name,Officename ( BO/SO/HO),Pincode,Sub-distname,Districtname,StateName
558410,Parrys,Chennai G.P.O.,600001,Fort - Tondiarpet,CHENNAI,TAMIL NADU
558411,Chennai,Anna Road H.O,600002,Egmore - Nungambakkam,CHENNAI,TAMIL NADU
558412,Parrys,Park Town H.O,600003,Fort - Tondiarpet,CHENNAI,TAMIL NADU
558413,Mylapore,Mylapore H.O,600004,Mylapore - Triplicane,CHENNAI,TAMIL NADU
558414,Tiruvallikkeni,Tiruvallikkeni S.O,600005,Mylapore - Triplicane,CHENNAI,TAMIL NADU


4. ### Postal code geocoding and distance calculations - pgeocode

* pgeocode is a Python library for high performance off-line querying of GPS coordinates, region name and municipality name from postal codes. 
* Distances between postal codes as well as general distance queries are also supported. 
* The used GeoNames database includes postal codes for 83 countries.

https://pgeocode.readthedocs.io

In [65]:
dist = pgeocode.GeoDistance('IN') # INDIA

In [74]:
# distance between two pincodes
dist.query_postal_code(600119, 600118) # retured distance in km

19.573262624518886

5. ### Distance between pincodes

In [71]:
data1[data1['Pincode']==600119]

,Village/Locality name,Officename ( BO/SO/HO),Pincode,Sub-distname,Districtname,StateName
558661,Sholinganallur,Sholinganallur S.O,600119,Sholinganallur,KANCHIPURAM,TAMIL NADU
558662,Uthandi,Sholinganallur S.O,600119,Tambaram,KANCHIPURAM,TAMIL NADU


In [142]:
data2 = data1

In [143]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1304 entries, 558410 to 601014
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Village/Locality name   1304 non-null   object
 1   Officename ( BO/SO/HO)  1304 non-null   object
 2   Pincode                 1304 non-null   int64 
 3   Sub-distname            1304 non-null   object
 4   Districtname            1304 non-null   object
 5   StateName               1304 non-null   object
dtypes: int64(1), object(5)
memory usage: 71.3+ KB


In [152]:
def distancebw(x):
    dist = pgeocode.GeoDistance('IN') # INDIA
    dist = dist.query_postal_code(600119, x.Pincode)
    return dist

In [162]:
data21 = data2.head(50)
data21

,Village/Locality name,Officename ( BO/SO/HO),Pincode,Sub-distname,Districtname,StateName
558410,Parrys,Chennai G.P.O.,600001,Fort - Tondiarpet,CHENNAI,TAMIL NADU
558411,Chennai,Anna Road H.O,600002,Egmore - Nungambakkam,CHENNAI,TAMIL NADU
558412,Parrys,Park Town H.O,600003,Fort - Tondiarpet,CHENNAI,TAMIL NADU
558413,Mylapore,Mylapore H.O,600004,Mylapore - Triplicane,CHENNAI,TAMIL NADU
558414,Tiruvallikkeni,Tiruvallikkeni S.O,600005,Mylapore - Triplicane,CHENNAI,TAMIL NADU
558415,NUNGAMBAKKAM,Greams Road S.O,600006,Egmore - Nungambakkam,CHENNAI,TAMIL NADU
558416,Vyasarpadi,Vepery S.O,600007,Fort - Tondiarpet,CHENNAI,TAMIL NADU
558417,Egmore,Egmore S.O,600008,Egmore - Nungambakkam,CHENNAI,TAMIL NADU
558418,Fort St George,Fort St George S.O,600009,Fort - Tondiarpet,CHENNAI,TAMIL NADU
558419,Kilpauk,Kilpauk S.O,600010,Perambur - Purasawakkam,CHENNAI,TAMIL NADU


In [163]:
data21['dis from A'] = data21.apply(distancebw,axis=1)

<ipython-input-163-c953ee84c92b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data21['dis from A'] = data21.apply(distancebw,axis=1)


In [166]:
data22 = data2

In [167]:
%time data22['dis from A'] = data22.apply(distancebw,axis=1)

Wall time: 7min 22s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [172]:
data22.to_csv('result.csv',index=False)

In [173]:
data22.head()

,Village/Locality name,Officename ( BO/SO/HO),Pincode,Sub-distname,Districtname,StateName,dis from A
558410,Parrys,Chennai G.P.O.,600001,Fort - Tondiarpet,CHENNAI,TAMIL NADU,23.079828
558411,Chennai,Anna Road H.O,600002,Egmore - Nungambakkam,CHENNAI,TAMIL NADU,20.959674
558412,Parrys,Park Town H.O,600003,Fort - Tondiarpet,CHENNAI,TAMIL NADU,21.121267
558413,Mylapore,Mylapore H.O,600004,Mylapore - Triplicane,CHENNAI,TAMIL NADU,21.121267
558414,Tiruvallikkeni,Tiruvallikkeni S.O,600005,Mylapore - Triplicane,CHENNAI,TAMIL NADU,19.498688
